In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import requests
import gzip
import json
import pandas as pd
import gensim
# URL of the JSON file
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz"

# Download the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to a file
    with open("/content/reviews_Sports_and_Outdoors_5.json.gz", "wb") as f:
        f.write(response.content)

    # Extract the gzip file
    with gzip.open("/content/reviews_Sports_and_Outdoors_5.json.gz", "rb") as f:
        data = [json.loads(line) for line in f]

    # Convert data to DataFrame
    df = pd.DataFrame(data)

    # Display the DataFrame
    print(df.head())
else:
    print("Failed to download the file.")


       reviewerID        asin                    reviewerName helpful  \
0    AIXZKN4ACSKI  1881509818                    David Briner  [0, 0]   
1  A1L5P841VIO02V  1881509818                 Jason A. Kramer  [1, 1]   
2   AB2W04NI4OEAD  1881509818                      J. Fernald  [2, 2]   
3  A148SVSWKTJKU6  1881509818  Jusitn A. Watts "Maverick9614"  [0, 0]   
4   AAAWJ6LW9WMOO  1881509818                    Material Man  [0, 0]   

                                          reviewText  overall  \
0  This came in on time and I am veru happy with ...      5.0   
1  I had a factory Glock tool that I was using fo...      5.0   
2  If you don't have a 3/32 punch or would like t...      4.0   
3  This works no better than any 3/32 punch you w...      4.0   
4  I purchased this thinking maybe I need a speci...      4.0   

                                   summary  unixReviewTime   reviewTime  
0                           Woks very good      1390694400  01 26, 2014  
1        Works as well

In [26]:
df.shape

(296337, 9)

In [33]:
df.reviewText[0]

'This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy'

In [57]:
reviewText=df.reviewText.apply(gensim.utils.simple_preprocess)
reviewText

0         [this, came, in, on, time, and, am, veru, happ...
1         [had, factory, glock, tool, that, was, using, ...
2         [if, you, don, have, punch, or, would, like, t...
3         [this, works, no, better, than, any, punch, yo...
4         [purchased, this, thinking, maybe, need, speci...
                                ...                        
296332    [this, is, water, bottle, done, right, it, is,...
296333    [if, you, re, looking, for, an, insulated, wat...
296334    [this, hydracentials, sporty, oz, double, insu...
296335    [as, usual, received, this, item, free, in, ex...
296336    [hydracentials, insulated, oz, water, bottle, ...
Name: reviewText, Length: 296337, dtype: object

In [58]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
    sorted_vocab=False

)


In [48]:
model.build_vocab(reviewText,progress_per=1000)

In [49]:
model.epochs

5

In [50]:
model.train(reviewText,total_examples=model.corpus_count,epochs=model.epochs)

(91343787, 121496535)

In [54]:
model.save("/content/drive/My Drive/word2vec_model.bin")

In [75]:
model_saved = Word2Vec.load("/content/drive/My Drive/word2vec_model.bin")
model_saved.wv.most_similar("sports")

[('sport', 0.7359817624092102),
 ('ymca', 0.6615786552429199),
 ('volleyball', 0.6418856382369995),
 ('events', 0.6269948482513428),
 ('racquetball', 0.6048140525817871),
 ('chalet', 0.6025053262710571),
 ('athletes', 0.5872717499732971),
 ('academy', 0.5871973633766174),
 ('sporting', 0.5845939517021179),
 ('divers', 0.5814198851585388)]

In [59]:
model_saved.wv.similarity(w1="received",w2="collected")

0.100263946

In [71]:
model_saved.wv.similarity(w1="amazing",w2="awesome")

0.8907093

In [60]:
model_saved.wv.similarity(w1="great",w2="product")

-0.11862085